In [1]:
import warnings
import os
import numpy as np
warnings.filterwarnings('ignore')
from sklearn.ensemble import ExtraTreesClassifier,VotingClassifier,RandomForestClassifier
from sklearn.model_selection import cross_val_score
import librosa
import pandas as pd

# Load Data

In [11]:
SAMPLE_RATE = 8000 # 8kHZ
DATA_PATH="./mypml/digit-prediction-2019/"

In [12]:
X_train = np.load(f'{DATA_PATH}x_train.npy') 
X_test = np.load(f'{DATA_PATH}x_test.npy')
y_train = np.load(f'{DATA_PATH}y_train.npy')

In [13]:
X = np.concatenate((X_train,X_test))
print(f'Train: {X_train.shape}\nTest: {X_test.shape}\nFull: {X.shape}')
train_size = X_train.shape[0]
del X_train,X_test

Train: (1000, 10000)
Test: (498, 10000)
Full: (1498, 10000)


In [14]:
gmms = [librosa.feature.mfcc(b, sr = SAMPLE_RATE, n_mfcc=20) for b in X]
res =[pd.Series(np.hstack((np.mean(gmm, axis=1), np.std(gmm, axis=1)))) for gmm in gmms]
len(res)

1498

# Model selection

In [ ]:
d_train,d_test=res[:train_size],res[train_size:]

In [ ]:
classifier = VotingClassifier(estimators=
                                  [('etc',ExtraTreesClassifier(
                                      criterion = 'entropy',
                                      n_estimators = 1000,
                                      max_features = 'auto',
                                      max_depth = 14,
                                      min_samples_leaf = 1,
                                      min_samples_split = 3,)),
                                    ('rfc',RandomForestClassifier(
                                        criterion='gini',
                                        n_estimators=1000,
                                        max_features='log2',
                                    max_depth=None))],voting="soft")

In [ ]:
cross_val_score(classifier,d_train,y_train,cv=10).mean()

# Submit

In [ ]:
classifier.fit(d_train,y_train)
preds = classifier.predict(d_test)

In [ ]:
with open('prediction.csv', 'w') as out:
    print('Id,Label', file=out)
    for pair in enumerate(preds, 1):
        print('%i,%i' % pair, file=out)